In [ ]:
pip install python-constraint

### Tasks for Artificial Intelligence

Scenario 1:

Suppose Ciara knows Python, and only has funds to hire three more people.

In [2]:
# Define the roles and the candidates with their qualifications
roles = ["Py_1", "Py_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
candidates = {
    "Ciara": ["Py_1"],
    "Peter": ["Py_2", "AI_1", "AI_2"],
    "Juan": ["Web", "AI_1", "AI_2"],
    "Jim": ["AI_1", "AI_2", "Systems"],
    "Jane": ["Py_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Py_2"],
    "Anita": ["Web", "AI_1", "AI_2"]
}

# Initialize variables to store the current solution and the list of solutions
current_option = {}
options = []

# Define the constraints
def is_valid(option):
    # Ciara must take Py_1
    if option.get("Py_1") != "Ciara":
        return False

    # A person can only occupy one of the two available AI positions
    ai_roles = [option.get("AI_1"), option.get("AI_2")]
    if len(set(ai_roles)) != 2:
        return False

    # Jane is the only one qualified to occupy the database position
    if option.get("Database") != "Jane":
        return False

    # Check if Ciara can hire 3 more people
    if len(set(current_option.values())) > 4:
        return False

    # Check if a person can only occupy a maximum of two roles
    candidates_count = {candidate: list(current_option.values()).count(candidate) for candidate in candidates.keys()}
    if any(count > 2 for count in candidates_count.values()):
        return False

    # Check if the individual is qualified for the roles they occupy
    for role, candidate in option.items():
        if role not in candidates[candidate]:
            return False

    return True

# Define the backtracking algorithm
def backtrack(role_index):
    if role_index == len(roles):
        # All roles are filled, check if the solution is valid
        if is_valid(current_option):
            options.append(dict(current_option))
        return

    current_role = roles[role_index]

    for candidate in candidates.keys():
        current_option[current_role] = candidate
        backtrack(role_index + 1)
        current_option[current_role] = None

# Run the backtracking algorithm
backtrack(0)

# Print the options by role
for i, option in enumerate(options):
    print(f"Option {i+1}:")
    for role, candidate in option.items():
        print(f"Role {role}: {candidate}")
    print()


Option 1:
Role Py_1: Ciara
Role Py_2: Jane
Role AI_1: Juan
Role AI_2: Jim
Role Web: Juan
Role Database: Jane
Role Systems: Jim

Option 2:
Role Py_1: Ciara
Role Py_2: Jane
Role AI_1: Jim
Role AI_2: Juan
Role Web: Juan
Role Database: Jane
Role Systems: Jim

Option 3:
Role Py_1: Ciara
Role Py_2: Jane
Role AI_1: Jim
Role AI_2: Anita
Role Web: Anita
Role Database: Jane
Role Systems: Jim

Option 4:
Role Py_1: Ciara
Role Py_2: Jane
Role AI_1: Anita
Role AI_2: Jim
Role Web: Anita
Role Database: Jane
Role Systems: Jim



Scenario 2:

Suppose Ciara and Juan become partners, with the additional funds they can now employ four more people but must employ another AI Engineer, so they need 2 Python Programmers, 3 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.


In [ ]:
from constraint import Problem

# Defines the roles and the candidates with their qualifications
roles = ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3", "Web", "Database", "Systems"]
candidates = {
    "Ciara": ["Py_1", "Py_2"],
    "Peter": ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3"],
    "Juan": ["Web", "AI_1", "AI_2", "AI_3"],
    "Jim": ["AI_1", "AI_2", "AI_3", "Systems"],
    "Jane": ["Py_1", "Py_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Py_1", "Py_2"],
    "Anita": ["Web", "AI_1", "AI_2", "AI_3"]
}

def create_problem(roles, candidates):
    problem = Problem()

    # Add variables (roles) and their domains (candidates who can take the roles)
    for role in roles:
        problem.addVariable(role, list(candidates.keys()))

    # Add constraints
    add_constraints(problem, roles, candidates)

    return problem

def add_constraints(problem, roles, candidates):
    # Add individual constraints
    problem.addConstraint(python_constraint, ["Py_1", "Py_2"])
    problem.addConstraint(ai_constraint, ["AI_1", "AI_2", "AI_3"])
    problem.addConstraint(db_constraint, ["Database"])
    problem.addConstraint(hiring_constraint, roles)
    problem.addConstraint(max_two_positions_constraint, roles)
    problem.addConstraint(qualification_constraint, roles)

def python_constraint(Py_1, Py_2):
    return Py_1 != Py_2

def ai_constraint(ai_1, ai_2, ai_3):
    return ai_1 != ai_2 and ai_1 != ai_3 and ai_2 != ai_3

def db_constraint(database):
    return database == "Jane"

def hiring_constraint(*args):
    candidates_set = set(args)
    if "Ciara" in candidates_set and "Juan" in candidates_set:
        return len(candidates_set) <= 6
    elif "Ciara" in candidates_set or "Juan" in candidates_set:
        return len(candidates_set) <= 5
    else:
        return len(candidates_set) == 4

def max_two_positions_constraint(*args):
    candidates_count = dict()
    for candidate in args:
        if candidate in candidates_count:
            candidates_count[candidate] += 1
        else:
            candidates_count[candidate] = 1
    return all(count <= 2 for count in candidates_count.values())

def qualification_constraint(*args):
    for role, candidate in zip(roles, args):
        if role not in candidates[candidate]:
            return False
    return True

def solve_and_print(problem, roles):
    options = problem.getSolutions()

    total_combinations = 0
    unique_options = []

    for option in options:
        # Post-process the solution as needed
        post_process_option(option, roles)

        # Adds to unique options if not already present
        if option not in unique_options:
            unique_options.append(option)
            total_combinations += 1

    # Prints the options by role
    for i, option in enumerate(unique_options):
        print_option(i + 1, option)

def post_process_option(option, roles):
    # For roles with the same qualifications, sorts the individuals
    ai_roles = sorted([option["AI_1"], option["AI_2"], option["AI_3"]])
    option["AI_1"], option["AI_2"], option["AI_3"] = ai_roles[0], ai_roles[1], ai_roles[2]

    python_roles = sorted([option["Py_1"], option["Py_2"]])
    option["Py_1"], option["Py_2"] = python_roles

def print_option(index, option):
    print(f"Option {index}:")
    for role, candidate in option.items():
        print(f"Role {role}: {candidate}")
    print()

# Original roles and candidates
roles = ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3", "Web", "Database", "Systems"]
candidates = {
    "Ciara": ["Py_1", "Py_2"],
    "Peter": ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3"],
    "Juan": ["Web", "AI_1", "AI_2", "AI_3"],
    "Jim": ["AI_1", "AI_2", "AI_3", "Systems"],
    "Jane": ["Py_1", "Py_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Py_1", "Py_2"],
    "Anita": ["Web", "AI_1", "AI_2", "AI_3"]
}

# Create and solve the problem
problem = create_problem(roles, candidates)
solve_and_print(problem, roles)


In [ ]:
from constraint import Problem

# Define the roles and the candidates with their qualifications
roles = ["py_1", "py_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
candidates = {
    "Ciara": ["py_1"],
    "Peter": ["py_2", "AI_1", "AI_2"],
    "Juan": ["Web", "AI_1", "AI_2"],
    "Jim": ["AI_1", "AI_2", "Systems"],
    "Jane": ["py_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "py_2"],
    "Anita": ["Web", "AI_1", "AI_2"]
}

def create_problem(roles, candidates):
    problem = Problem()

    # Add variables (roles) and their domains (candidates who can take the roles)
    for role in roles:
        problem.addVariable(role, list(candidates.keys()))

    # Add constraints
    add_constraints(problem, roles, candidates)

    return problem

def add_constraints(problem, roles, candidates):
    # Add individual constraints
    problem.addConstraint(python_constraint, ["py_1", "py_2"])
    problem.addConstraint(ai_constraint, ["AI_1", "AI_2"])
    problem.addConstraint(db_constraint, ["Database"])
    problem.addConstraint(hiring_constraint, roles)
    problem.addConstraint(max_two_positions_constraint, roles)
    problem.addConstraint(qualification_constraint, roles)

def python_constraint(py_1, py_2):
    return py_1 != py_2

def ai_constraint(ai_1, ai_2):
    return ai_1 != ai_2

def db_constraint(database):
    return database == "Jane"

def hiring_constraint(*args):
    candidates_set = set(args)
    if "Ciara" in candidates_set and "Juan" in candidates_set:
        return len(candidates_set) <= 6
    elif "Ciara" in candidates_set or "Juan" in candidates_set:
        return len(candidates_set) <= 5
    else:
        return len(candidates_set) == 4

def max_two_positions_constraint(*args):
    candidates_count = dict()
    for candidate in args:
        if candidate in candidates_count:
            candidates_count[candidate] += 1
        else:
            candidates_count[candidate] = 1
    return all(count <= 2 for count in candidates_count.values())

def qualification_constraint(*args):
    for role, candidate in zip(roles, args):
        if role not in candidates[candidate]:
            return False
    return True

def solve_and_print(problem, roles):
    options = problem.getSolutions()

    total_combinations = 0
    unique_options = []

    candidates_count = dict()  # Track the count of each candidate

    for option in options:
        # Post-process the solution as needed
        post_process_option(option, roles)

        # Adds to unique options if not already present
        if option not in unique_options:
            unique_options.append(option)
            total_combinations += 1

            # Update the count of each candidate
            for candidate in option.values():
                if candidate in candidates_count:
                    candidates_count[candidate] += 1
                else:
                    candidates_count[candidate] = 1

    # Prints the options by role
    for i, option in enumerate(unique_options):
        print_option(i + 1, option)

    # Print the number of solutions
    print(f"\nTotal number of solutions: {total_combinations}")

    # Print the count of each candidate
    print("\nCount of each candidate:")
    for candidate, count in candidates_count.items():
        print(f"{candidate}: {count}")

def post_process_option(option, roles):
    # For roles with the same qualifications, sorts the individuals
    ai_roles = sorted([option["AI_1"], option["AI_2"]])
    option["AI_1"], option["AI_2"] = ai_roles[0], ai_roles[1]

    python_roles = sorted([option["py_1"], option["py_2"]])
    option["py_1"], option["py_2"] = python_roles

def print_option(index, option):
    print(f"\nOption {index}:")
    for role, candidate in option.items():
        print(f"Role {role}: {candidate}")

# Create and solve the problem
problem = create_problem(roles, candidates)
solve_and_print(problem, roles)


These problems be solved using several other algorithm’s we have studied in the module. Choose one of these algorithms and discuss your answer in detail including a proof of your hypothesis in code .

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, PULP_CBC_CMD

# Define roles and candidates
roles = ["py_1", "py_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
candidates = ["Peter", "Jim", "Jane", "Mary", "Bruce", "Anita"]

# Create ILP problem
prob = LpProblem("CSP_ILP", LpMinimize)

# Define binary variables
x = {(i, j): LpVariable(name=f"x_{i}_{j}", cat="Binary") for i in roles for j in candidates}

# Objective function (minimize the sum of variables)
prob += lpSum(x[i, j] for i in roles for j in candidates)

# Constraints
for i in roles:
    prob += lpSum(x[i, j] for j in candidates) == 1

for j in candidates:
    prob += lpSum(x[i, j] for i in roles) <= 3  # Relax the maximum positions a candidate can take

# Qualification constraints
prob += lpSum(x["py_1", j] for j in candidates) == 1
prob += lpSum(x["py_2", j] for j in candidates) == 1
prob += lpSum(x["AI_1", j] for j in candidates) == 1
prob += lpSum(x["AI_2", j] for j in candidates) == 1
prob += lpSum(x["Database", j] for j in candidates) == 1
prob += lpSum(x["Systems", j] for j in candidates) == 1

# Additional hiring requirements
prob += lpSum(x[i, j] for i in ["py_1", "py_2"] for j in candidates) == 2  # 2 Python Programmers
prob += lpSum(x[i, j] for i in ["AI_1", "AI_2"] for j in candidates) == 3  # 3 AI Engineers

# 1 Web Designer
for j in candidates:
    prob += x["Web", j] == 1

# 1 Database Admin
for j in candidates:
    prob += x["Database", j] == 1

# 1 Systems Engineer
for j in candidates:
    prob += x["Systems", j] == 1

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=False))

# Print the options for each candidate
print("Options for each candidate:")
for j in candidates:
    options = [i for i in roles if value(x[i, j]) >= 0.5]
    print(f"{j}: {options}")


### Tasks for Data Visualisation

In [20]:
import pandas as pd
from IPython.display import display

# ... (your existing code)

# Create DataFrames
roles_df = pd.DataFrame({"Roles": roles})
candidates_df = pd.DataFrame(list(candidates.items()), columns=["Candidate", "Abilities"])

# Display DataFrames using IPython display
print("\nTable of available Roles:")
display(roles_df)

print("\nTable of candidates qualified:")
display(candidates_df)



Table of available Roles:


,Roles
0,Py_1
1,Py_2
2,AI_1
3,AI_2
4,AI_3
5,Web
6,Database
7,Systems



Table of candidates qualified:


,Candidate,Abilities
0,Ciara,"[Py_1, Py_2]"
1,Peter,"[Py_1, Py_2, AI_1, AI_2, AI_3]"
2,Juan,"[Web, AI_1, AI_2, AI_3]"
3,Jim,"[AI_1, AI_2, AI_3, Systems]"
4,Jane,"[Py_1, Py_2, Database]"
5,Mary,"[Web, Systems]"
6,Bruce,"[Systems, Py_1, Py_2]"
7,Anita,"[Web, AI_1, AI_2, AI_3]"


In [19]:
def show_solutions_gui(options, candidate_hire_count):
    root = tk.Tk()
    root.title("Constraint Problem Solutions")

    # Function to format option for display
    def format_option(option):
        formatted_option = ""
        for role, candidate in option.items():
            formatted_option += f"Role {role}: {candidate}\n"
        return formatted_option

    # Listbox to display options
    listbox = tk.Listbox(root, selectmode=tk.SINGLE)
    listbox.pack(pady=10)

    for i, option in enumerate(options):
        formatted_option = format_option(option)
        listbox.insert(tk.END, formatted_option)

    # Label for candidate hire counts
    hire_counts_label = tk.Label(root, text="Candidate Hire Counts:")
    hire_counts_label.pack(pady=10)

    # Function to update chart on listbox selection
    def update_chart(selected_option_index):
        if not selected_option_index:
            return

        selected_option = candidate_hire_count[selected_option_index[0]]

        fig = go.Figure()
        for candidate, count in selected_option.items():
            fig.add_trace(go.Bar(x=[candidate], y=[count], name=f'{candidate} Hire Count'))

        fig.update_layout(
            title='Candidate Hire Counts for Selected Option',
            xaxis_title='Candidate',
            yaxis_title='Hire Count',
            barmode='stack'
        )

        fig.show()

    listbox.bind("<ButtonRelease-1>", lambda event: update_chart(listbox.curselection()))

    root.mainloop()

# Display solutions and candidate hire counts in GUI
show_solutions_gui(options, candidate_hire_count)


In [21]:
import tkinter as tk
from tkinter import ttk

# Define the roles and the candidates with their qualifications
roles = ["Py_1", "Py_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
candidates = {
    "Ciara": ["Py_1"],
    "Peter": ["Py_2", "AI_1", "AI_2"],
    "Juan": ["Web", "AI_1", "AI_2"],
    "Jim": ["AI_1", "AI_2", "Systems"],
    "Jane": ["Py_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Py_2"],
    "Anita": ["Web", "AI_1", "AI_2"]
}

# Initialize variables to store the current solution and the list of solutions
current_option = {}
options = []

# Define the constraints
def is_valid(option):
    # Ciara must take Py_1
    if option.get("Py_1") != "Ciara":
        return False

    # A person can only occupy one of the two available AI positions
    ai_roles = [option.get("AI_1"), option.get("AI_2")]
    if len(set(ai_roles)) != 2:
        return False

    # Jane is the only one qualified to occupy the database position
    if option.get("Database") != "Jane":
        return False

    # Check if Ciara can hire 3 more people
    if len(set(current_option.values())) > 4:
        return False

    # Check if a person can only occupy a maximum of two roles
    candidates_count = {candidate: list(current_option.values()).count(candidate) for candidate in candidates.keys()}
    if any(count > 2 for count in candidates_count.values()):
        return False

    # Check if the individual is qualified for the roles they occupy
    for role, candidate in option.items():
        if role not in candidates[candidate]:
            return False

    return True

# Define the backtracking algorithm
def backtrack(role_index):
    if role_index == len(roles):
        # All roles are filled, check if the solution is valid
        if is_valid(current_option):
            options.append(dict(current_option))
        return

    current_role = roles[role_index]

    for candidate in candidates.keys():
        current_option[current_role] = candidate
        backtrack(role_index + 1)
        current_option[current_role] = None

# Run the backtracking algorithm
backtrack(0)

# GUI
def show_candidates(event):
    selected_option_index = listbox.curselection()
    
    if not selected_option_index:
        return
    
    selected_option = int(selected_option_index[0]) + 1
    option_label.config(text=f"Option {selected_option} Candidates:")
    
    for i, role in enumerate(roles):
        candidate_name = options[selected_option - 1][role]
        candidates_label[i].config(text=f"Role {role}: {candidate_name}")

root = tk.Tk()
root.title("Candidates by Option")

listbox = tk.Listbox(root)
for i in range(1, len(options) + 1):
    listbox.insert(tk.END, f"Option {i}")
listbox.pack(pady=10)
listbox.bind("<ButtonRelease-1>", show_candidates)

option_label = tk.Label(root, text="")
option_label.pack(pady=10)

candidates_label = []
for role in roles:
    label = tk.Label(root, text="")
    candidates_label.append(label)
    label.pack()

root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
from constraint import Problem

# Define the roles and the candidates with their qualifications
roles = ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3", "Web", "Database", "Systems"]
candidates = {
    "Ciara": ["Py_1", "Py_2"],
    "Peter": ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3"],
    "Juan": ["Web", "AI_1", "AI_2", "AI_3"],
    "Jim": ["AI_1", "AI_2", "AI_3", "Systems"],
    "Jane": ["Py_1", "Py_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Py_1", "Py_2"],
    "Anita": ["Web", "AI_1", "AI_2", "AI_3"]
}

# Original roles and candidates
original_roles = ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3", "Web", "Database", "Systems"]
original_candidates = {
    "Ciara": ["Py_1", "Py_2"],
    "Peter": ["Py_1", "Py_2", "AI_1", "AI_2", "AI_3"],
    "Juan": ["Web", "AI_1", "AI_2", "AI_3"],
    "Jim": ["AI_1", "AI_2", "AI_3", "Systems"],
    "Jane": ["Py_1", "Py_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Py_1", "Py_2"],
    "Anita": ["Web", "AI_1", "AI_2", "AI_3"]
}

def create_problem(roles, candidates):
    problem = Problem()

    # Add variables (roles) and their domains (candidates who can take the roles)
    for role in roles:
        problem.addVariable(role, list(candidates.keys()))

    # Add constraints
    add_constraints(problem, roles, candidates)

    return problem

def add_constraints(problem, roles, candidates):
    # Add individual constraints
    problem.addConstraint(python_constraint, ["Py_1", "Py_2"])
    problem.addConstraint(ai_constraint, ["AI_1", "AI_2", "AI_3"])
    problem.addConstraint(db_constraint, ["Database"])
    problem.addConstraint(hiring_constraint, roles)
    problem.addConstraint(max_two_positions_constraint, roles)
    problem.addConstraint(qualification_constraint, roles)

def python_constraint(Py_1, Py_2):
    return Py_1 != Py_2

def ai_constraint(ai_1, ai_2, ai_3):
    return ai_1 != ai_2 and ai_1 != ai_3 and ai_2 != ai_3

def db_constraint(database):
    return database == "Jane"

def hiring_constraint(*args):
    candidates_set = set(args)
    if "Ciara" in candidates_set and "Juan" in candidates_set:
        return len(candidates_set) <= 6
    elif "Ciara" in candidates_set or "Juan" in candidates_set:
        return len(candidates_set) <= 5
    else:
        return len(candidates_set) == 4

def max_two_positions_constraint(*args):
    candidates_count = dict()
    for candidate in args:
        if candidate in candidates_count:
            candidates_count[candidate] += 1
        else:
            candidates_count[candidate] = 1
    return all(count <= 2 for count in candidates_count.values())

def qualification_constraint(*args):
    for role, candidate in zip(original_roles, args):
        if role not in original_candidates[candidate]:
            return False
    return True

def solve_and_print(problem, roles):
    options = problem.getSolutions()

    total_combinations = 0
    unique_options = []

    candidate_hire_count = {candidate: 0 for candidate in original_candidates.keys()}

    for option in options:
        # Post-process the solution as needed
        post_process_option(option, roles)

        # Count how many times each candidate could be hired
        for candidate in option.values():
            candidate_hire_count[candidate] += 1

        # Adds to unique options if not already present
        if option not in unique_options:
            unique_options.append(option)
            total_combinations += 1

    # Prints the options by role
    for i, option in enumerate(unique_options):
        print_option(i + 1, option)

    print("\nCandidate Hire Counts:")
    for candidate, count in candidate_hire_count.items():
        print(f"{candidate}: {count} times")

    # Return options and hire counts for GUI display
    return unique_options, candidate_hire_count

def post_process_option(option, roles):
    # For roles with the same qualifications, sorts the individuals
    ai_roles = sorted([option["AI_1"], option["AI_2"], option["AI_3"]])
    option["AI_1"], option["AI_2"], option["AI_3"] = ai_roles[0], ai_roles[1], ai_roles[2]

    python_roles = sorted([option["Py_1"], option["Py_2"]])
    option["Py_1"], option["Py_2"] = python_roles

def print_option(index, option):
    print(f"Option {index}:")
    for role, candidate in option.items():
        print(f"Role {role}: {candidate}")
    print()

# GUI for displaying solutions and candidate hire counts
def show_solutions_gui(options, candidate_hire_count):
    root = tk.Tk()
    root.title("Solutions and Candidate Hire Counts")

    listbox = tk.Listbox(root)
    for i, option in enumerate(options, start=1):
        listbox.insert(tk.END, f"Option {i}")
    listbox.pack(pady=10)

    def show_solution(event):
        selected_option_index = listbox.curselection()

        if not selected_option_index:
            return

        selected_option = options[int(selected_option_index[0])]
        option_label.config(text=f"Selected Option:\n{format_option(selected_option)}")

    listbox.bind("<ButtonRelease-1>", show_solution)

    option_label = tk.Label(root, text="")
    option_label.pack(pady=10)

    hire_counts_label = tk.Label(root, text="Candidate Hire Counts:")
    hire_counts_label.pack(pady=10)

    for candidate, count in candidate_hire_count.items():
        label = tk.Label(root, text=f"{candidate}: {count} times")
        label.pack()

    root.mainloop()

def format_option(option):
    formatted_option = ""
    for role, candidate in option.items():
        formatted_option += f"Role {role}: {candidate}\n"
    return formatted_option

# Create and solve the problem
problem = create_problem(roles, candidates)
options, candidate_hire_count = solve_and_print(problem, roles)

# Display solutions and candidate hire counts in GUI
show_solutions_gui(options, candidate_hire_count)


Option 1:
Role AI_1: Anita
Role AI_2: Jim
Role AI_3: Juan
Role Py_1: Bruce
Role Py_2: Ciara
Role Database: Jane
Role Systems: Bruce
Role Web: Juan

Option 2:
Role AI_1: Anita
Role AI_2: Jim
Role AI_3: Juan
Role Py_1: Bruce
Role Py_2: Ciara
Role Database: Jane
Role Systems: Bruce
Role Web: Anita

Option 3:
Role AI_1: Anita
Role AI_2: Jim
Role AI_3: Juan
Role Py_1: Bruce
Role Py_2: Ciara
Role Database: Jane
Role Systems: Jim
Role Web: Juan

Option 4:
Role AI_1: Anita
Role AI_2: Jim
Role AI_3: Juan
Role Py_1: Bruce
Role Py_2: Ciara
Role Database: Jane
Role Systems: Jim
Role Web: Anita

Option 5:
Role AI_1: Anita
Role AI_2: Jim
Role AI_3: Juan
Role Py_1: Bruce
Role Py_2: Jane
Role Database: Jane
Role Systems: Bruce
Role Web: Juan

Option 6:
Role AI_1: Anita
Role AI_2: Jim
Role AI_3: Juan
Role Py_1: Bruce
Role Py_2: Jane
Role Database: Jane
Role Systems: Bruce
Role Web: Anita

Option 7:
Role AI_1: Anita
Role AI_2: Jim
Role AI_3: Juan
Role Py_1: Bruce
Role Py_2: Jane
Role Database: Jane
Role